In [1]:
!pip install --upgrade pip
!pip install paddlepaddle -f https://www.paddlepaddle.org.cn/whl/linux/cpu/avx/stable.html
!pip install numpy==1.26.4 pandas==2.2.2 opencv-python-headless paddleocr ultralytics
!pip install -q ultralytics paddleocr opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in links: https://www.paddlepaddle.org.cn/whl/linux/cpu/avx/stable.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 MB 67.7 MB/s eta 0:00:00
  Attempting uninstall: opt_einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [paddlepaddle]
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of paddlex to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━

In [2]:

!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d piotrstefaskiue/poland-vehicle-license-plate-dataset
!unzip -qo poland-vehicle-license-plate-dataset.zip -d /content/dataset


Dataset URL: https://www.kaggle.com/datasets/piotrstefaskiue/poland-vehicle-license-plate-dataset
License(s): other
 97% 459M/474M [00:04<00:00, 109MB/s]
100% 474M/474M [00:04<00:00, 106MB/s]


In [3]:

import contextlib
import os
import sys
import time
import cv2
import xml.etree.ElementTree as ET
from paddleocr import PaddleOCR
from ultralytics import YOLO
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from typing import List, Tuple, Dict
import glob
import pandas as pd
from math import log
import random
from PIL import Image
import os, time, cv2, glob, xml.etree.ElementTree as ET
import numpy as np, pandas as pd
from tqdm import tqdm

/usr/local/lib/python3.11/dist-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:

# ------------------
def normalize(text: str) -> str:
    return text.replace(" ", "").replace("-", "").replace("=", "").upper()

def compute_iou(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    areaA = (boxA[2]-boxA[0]) * (boxA[3]-boxA[1])
    areaB = (boxB[2]-boxB[0]) * (boxB[3]-boxB[1])
    return interArea / (areaA + areaB - interArea + 1e-6)

def calculate_final_grade(acc, t):
    if acc < 60 or t > 60:
        return 2.0
    a_norm = (acc - 60) / 40
    t_norm = (60 - t) / 50
    score = 0.7 * a_norm + 0.3 * t_norm
    return round((2.0 + 3.0 * score) * 2) / 2

# ------------------------------ dataset
images_path = "/content/dataset/photos"
ann_file = "/content/dataset/annotations.xml"

tree = ET.parse(ann_file)
root = tree.getroot()
data = []
for image in root.findall("image"):
    fn = image.attrib.get("name")
    for box in image.findall("box"):
        if box.attrib.get("label") == "plate":
            xtl = float(box.attrib.get("xtl"))
            ytl = float(box.attrib.get("ytl"))
            xbr = float(box.attrib.get("xbr"))
            ybr = float(box.attrib.get("ybr"))
            text = ""
            for att in box.findall("attribute"):
                if att.attrib.get("name") == "plate number":
                    text = att.text.strip() if att.text else ""
            data.append([fn, xtl, ytl, xbr, ybr, text])

df = pd.DataFrame(data, columns=["filename", "xmin", "ymin", "xmax", "ymax", "plate_text"])
files = df["filename"].unique()

# -------------------------- losowe
if len(files) >= 100:
    files = np.random.choice(files, 100, replace=False)


# ------------------------------ modelelele
yolo = YOLO("/content/license_plate_detector.pt")
ocr = PaddleOCR(use_angle_cls=True, lang="en", use_gpu=True)

# ------------------------------
correct, iou_list = 0, []
start = time.time()

for f in files:

    img_path = os.path.join(images_path, f)
    img = cv2.imread(img_path)
    if img is None:
        continue

    #--- yolo
    res = yolo(img)
    if res and res[0].boxes is not None and len(res[0].boxes) > 0:
        boxes = res[0].boxes.data.cpu().numpy()
        best_box = max(boxes, key=lambda b: b[4])
        pred_box = [int(best_box[i]) for i in range(4)]
    else:
        pred_box = None

    # ------------------------------ gt z annotations
    gt = df[df["filename"] == f].iloc[0]
    gt_box = [int(gt["xmin"]), int(gt["ymin"]), int(gt["xmax"]), int(gt["ymax"])]
    gt_text = normalize(gt["plate_text"])  # normalizacja ground truth

    # ----------------------
    recog = ""
    if pred_box is not None: # crop
        crop = img[pred_box[1]:pred_box[3], pred_box[0]:pred_box[2]]
        try:
            crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        except:
            crop_rgb = crop

        ocr_res = ocr.ocr(crop_rgb, cls=True)

        # ---- wybor na podstawie wielkosci tekstu
        if (ocr_res and isinstance(ocr_res, list) and len(ocr_res) > 0 and isinstance(ocr_res[0], list)):
            best_candidate = None
            max_area = 0
            min_confidence = 0.5  # minimalna ufność
            for line in ocr_res[0]:
                if not (isinstance(line, list) and len(line) >= 2 and isinstance(line[1], tuple)):
                    continue
                coords = line[0]
                text, conf = line[1]
                if conf < min_confidence:
                    continue

                xs = [pt[0] for pt in coords]
                ys = [pt[1] for pt in coords]
                x_min, x_max = min(xs), max(xs)
                y_min, y_max = min(ys), max(ys)
                area = (x_max - x_min) * (y_max - y_min)

                if area > max_area:
                    max_area = area
                    best_candidate = normalize(text)

            if best_candidate:
                recog = best_candidate
            else:
                # else wybór po max ufnosci
                candidates = []
                for line in ocr_res[0]:
                    if not (isinstance(line, list) and len(line) >= 2 and isinstance(line[1], tuple)):
                        continue
                    txt, conf = line[1]
                    candidates.append((normalize(txt), conf))
                if candidates:
                    recog = max(candidates, key=lambda x: x[1])[0]
                else:
                    recog = ""
        else:
            recog = ""

        # ----- zliczanie
        if recog == gt_text:
            correct += 1
        iou_list.append(compute_iou(pred_box, gt_box))
    else:
        iou_list.append(0.0)

# ------------------------------
t = time.time() - start
acc = (correct / len(files)) * 100
grade = calculate_final_grade(acc, t)

print(f"liczba obrazow: {len(files)}")
print(f"% dokładności: {acc:.2f}%")
print(f"czas: {t:.2f} s")
print(f"ocena: {grade}")


[2025/06/17 09:03:38] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_l